## Singular Value Decomposition (SVD)_pf

In [1]:
# Instale a biblioteca Surprise antes de executar este código: pip install scikit-surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
#df_pf = pd.read_csv('PF_tratados.csv')
df = pd.read_csv('/home/hub/Área de Trabalho/dados/PF_tratados.csv')
df.head()


,ID Cliente,Linha de Ação,Produto GPOM,Data de Venda,Classe de Serviço,Data de Nascimento,Gênero,Cidade,Microrregião,Estado,Fim de Vigência,Tipo Pessoa,Área de Atuação,Área de Atuação e Produto GPOM,Idade,Tempo de Contrato,Longitude,Latitude
0,56943,EDUCACAO,COLEGIO SESI NOVO ENSINO MEDIO,2022-10-28,ENSINO MEDIO – ITINERARIOS,2008-05-16,MASCULINO,CURITIBA,CURITIBA,PR,2023-12-15,FÍSICA,NÃO INFORMADO,COLEGIO SESI NOVO ENSINO MEDIO / NÃO INFORMADO,14,413,-49.264622,-25.419547
1,19906,EDUCACAO,EDUCACAO INFANTIL,2023-01-13,EDUCACAO INFANTIL - PRE-ESCOLA,1981-08-13,FEMININO,CURITIBA,CURITIBA,PR,2023-12-25,FÍSICA,NÃO INFORMADO,EDUCACAO INFANTIL / NÃO INFORMADO,41,346,-49.264622,-25.419547
2,19906,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2021-10-07,ENSINO FUNDAMENTAL - 1º AO 5º ANO,1981-08-13,FEMININO,CURITIBA,CURITIBA,PR,2022-12-14,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,40,433,-49.264622,-25.419547
3,19906,EDUCACAO,ENSINO FUNDAMENTAL INTERNACIONAL,2023-01-13,ENSINO FUNDAMENTAL - 1º AO 5º ANO,1981-08-13,FEMININO,CURITIBA,CURITIBA,PR,2023-12-25,FÍSICA,NÃO INFORMADO,ENSINO FUNDAMENTAL INTERNACIONAL / NÃO INFORMADO,41,346,-49.264622,-25.419547
4,6940,EDUCACAO,COLEGIO SESI INTERNACIONAL,2021-10-14,ENSINO MEDIO,2006-11-01,FEMININO,CURITIBA,CURITIBA,PR,2022-12-14,FÍSICA,NÃO INFORMADO,COLEGIO SESI INTERNACIONAL / NÃO INFORMADO,14,426,-49.264622,-25.419547


## Pré-processamento

In [3]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)

print("Variáveis categóricas na base de dados:", object_cols)


Variáveis categóricas na base de dados: ['Linha de Ação', 'Produto GPOM', 'Data de Venda', 'Classe de Serviço', 'Data de Nascimento', 'Gênero', 'Cidade', 'Microrregião', 'Estado', 'Fim de Vigência', 'Tipo Pessoa', 'Área de Atuação', 'Área de Atuação e Produto GPOM']


In [4]:
LE=LabelEncoder()
for i in object_cols:
    df[i]=df[[i]].apply(LE.fit_transform)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83872 entries, 0 to 83871
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID Cliente                      83872 non-null  int64  
 1   Linha de Ação                   83872 non-null  int64  
 2   Produto GPOM                    83872 non-null  int64  
 3   Data de Venda                   83872 non-null  int64  
 4   Classe de Serviço               83872 non-null  int64  
 5   Data de Nascimento              83872 non-null  int64  
 6   Gênero                          83872 non-null  int64  
 7   Cidade                          83872 non-null  int64  
 8   Microrregião                    83872 non-null  int64  
 9   Estado                          83872 non-null  int64  
 10  Fim de Vigência                 83872 non-null  int64  
 11  Tipo Pessoa                     83872 non-null  int64  
 12  Área de Atuação                 

In [6]:
df.head()

,ID Cliente,Linha de Ação,Produto GPOM,Data de Venda,Classe de Serviço,Data de Nascimento,Gênero,Cidade,Microrregião,Estado,Fim de Vigência,Tipo Pessoa,Área de Atuação,Área de Atuação e Produto GPOM,Idade,Tempo de Contrato,Longitude,Latitude
0,56943,0,10,562,15,12147,1,140,10,16,972,0,17,40,14,413,-49.264622,-25.419547
1,19906,0,17,626,9,3075,0,140,10,16,982,0,17,47,41,346,-49.264622,-25.419547
2,19906,0,20,237,12,3075,0,140,10,16,634,0,17,50,40,433,-49.264622,-25.419547
3,19906,0,20,626,12,3075,0,140,10,16,982,0,17,50,41,346,-49.264622,-25.419547
4,6940,0,8,242,14,11593,0,140,10,16,634,0,17,38,14,426,-49.264622,-25.419547


## Fazer previsões para recomendações futuras:

In [7]:
from surprise.model_selection import train_test_split
from surprise import SVD, Dataset, Reader

df['Data de Venda'] = pd.to_numeric(df['Data de Venda'])
reader = Reader(rating_scale=(0, 10))  # Ajuste a escala conforme necessário
data = Dataset.load_from_df(df[['ID Cliente', 'Produto GPOM', 'Data de Venda']], reader)

df['Produto GPOM'] = df['Produto GPOM'].astype('category')

# Divida o conjunto de dados em treino e teste
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Usa o algoritmo SVD (Singular Value Decomposition)
model = SVD()
model.fit(trainset)

# Cria uma lista para armazenar as recomendações de todos os usuários
all_user_recommendations = []

# Itere sobre todos os IDs de cliente no conjunto de dados
for user_id in df['ID Cliente'].unique():
    # Obtem as previsões para o usuário atual
    user_predictions = model.test([(str(user_id), iid, 0) for iid in df['Produto GPOM'].cat.codes.unique()])
    
    # Obtem as top N recomendações para o usuário atual (por exemplo, top 3)
    top_n = 3
    top_recommended = sorted(user_predictions, key=lambda x: x.est, reverse=True)[:top_n]
    
    # Armazena as recomendações na lista geral
    all_user_recommendations.extend([(user_id, item.iid, item.est) for item in top_recommended])

# Converte a lista de recomendações em um DataFrame
columns = ['ID Cliente', 'Item Recomendado', 'Estimativa de Avaliação']
df_recommendations = pd.DataFrame(all_user_recommendations, columns=columns)

# Adiciona a coluna 'NomeProduto' às recomendações
df_recommendations['NomeProduto'] = df_recommendations['Item Recomendado'].map(
    dict(zip(df['Produto GPOM'].cat.codes, df['Produto GPOM']))
)

print(df_recommendations[['ID Cliente', 'NomeProduto', 'Estimativa de Avaliação']])

#30 primeiros resultados
print(df_recommendations[['ID Cliente', 'NomeProduto', 'Estimativa de Avaliação']].head(30))


       ID Cliente  NomeProduto  Estimativa de Avaliação
0           56943           10                       10
1           56943           17                       10
2           56943           20                       10
3           19906           10                       10
4           19906           17                       10
...           ...          ...                      ...
41659       24880           17                       10
41660       24880           20                       10
41661        1045           10                       10
41662        1045           17                       10
41663        1045           20                       10

[41664 rows x 3 columns]
    ID Cliente  NomeProduto  Estimativa de Avaliação
0        56943           10                       10
1        56943           17                       10
2        56943           20                       10
3        19906           10                       10
4        19906           17          

**Utiliza o algoritmo KNN para gerar recomendações com base na similaridade entre os itens comprados anteriormente pelos clientes**

In [8]:
from surprise.model_selection import train_test_split
from surprise import KNNBasic, Dataset, Reader
import pandas as pd

df['Data de Venda'] = pd.to_numeric(df['Data de Venda'])

reader = Reader(rating_scale=(0, 10))  # Ajuste a escala conforme necessário

data = Dataset.load_from_df(df[['ID Cliente', 'Produto GPOM', 'Data de Venda']], reader)

# Divide o conjunto de dados em treino e teste
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Use o algoritmo KNNBasic para filtragem colaborativa baseada em itens
model = KNNBasic(sim_options={'user_based': False})  # Usamos item-based aqui
model.fit(trainset)

# Cria uma lista para armazenar as recomendações de todos os usuários
all_user_recommendations = []

# Itere sobre todos os IDs de cliente no conjunto de dados
for user_id in df['ID Cliente'].unique():
    # Obtenha os itens comprados anteriormente pelo cliente
    items_purchased = df[df['ID Cliente'] == user_id]['Produto GPOM'].tolist()

    # Calcula as similaridades entre os itens comprados
    similarities = []
    for item in items_purchased:
        # Verifica se o índice do item está dentro dos limites da matriz de similaridade
        if item in model.trainset.all_items():
            item_neighbors = model.get_neighbors(item, k=3)  
            similarities.extend([(item, neighbor, sim) for neighbor, sim in zip(item_neighbors, model.sim[item, item_neighbors])])

    # Seleciona os itens mais semelhantes
    top_similar_items = sorted(similarities, key=lambda x: x[2], reverse=True)[:3]

    # Armazena as recomendações na lista geral
    all_user_recommendations.extend([(user_id, item, 0) for _, item, _ in top_similar_items])

# Converte a lista de recomendações em um DataFrame
columns = ['ID Cliente', 'Item Recomendado', 'Estimativa de Avaliação']
df_recommendations = pd.DataFrame(all_user_recommendations, columns=columns)

# Adiciona a coluna 'NomeProduto' às recomendações
df_recommendations['NomeProduto'] = df_recommendations['Item Recomendado'].map(
    dict(zip(df['Produto GPOM'], df['Produto GPOM']))
)

print(df_recommendations[['ID Cliente', 'NomeProduto', 'Estimativa de Avaliação']])

#30 primeiros resultados
print(df_recommendations[['ID Cliente', 'NomeProduto', 'Estimativa de Avaliação']].head(30))



Computing the msd similarity matrix...
Done computing similarity matrix.
       ID Cliente  NomeProduto  Estimativa de Avaliação
0           56943           24                        0
1           56943           24                        0
2           56943            7                        0
3           19906            6                        0
4           19906            6                        0
...           ...          ...                      ...
41659       24880            7                        0
41660       24880           10                        0
41661        1045           27                        0
41662        1045           23                        0
41663        1045           16                        0

[41664 rows x 3 columns]
    ID Cliente  NomeProduto  Estimativa de Avaliação
0        56943           24                        0
1        56943           24                        0
2        56943            7                        0
3        19906   